In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음.DF로 로드.

feature_name_df = pd.read_csv('features.txt', sep='\s+',
                              header = None, names=['co